# Baden-Württemberg

Every federal state is represented by its own input directory and is processed into a NUTS level 2 directory containing a sub-folder for each discharge location. These folder names are derived from NUTS and reflect the CAMELS id. The NUTS level 2 code for Baden-Württemberg is `DE1`.

To pre-process the data, you need to write (at least) two functions. One should extract all metadata and condense it into a single `pandas.DataFrame`. This is used to build the folder structure and derive the ids.
The second function has to take an id, as provided by the state authorities, called `provider_id` and return a `pandas.DataFrame` with the transformed data. The dataframe needs the three columns `['date', 'q' | 'w', 'flag']`.

For easier and unified output handling, the `camelsp` package contains a context object called `Bundesland`. It takes a number of names and abbreviations to identify the correct federal state and returns an object that holds helper and save functions.

The context saves files as needed and can easily be changed to save files with different strategies, ie. fill missing data with NaN, merge data into a single file, create files for each variable or pack everything together into a netcdf.

In [1]:
import pandas as pd
import os
from tqdm import tqdm
from typing import Union, Dict
import zipfile
from datetime import datetime as dt

from camelsp import Bundesland

The context can also be instantiated as any regular Python class, ie. to load only the default input data path, that we will user later.

In [2]:
# the context also makes the input path available, if camelsp was install locally
BASE = Bundesland('bw').input_path
BASE

'/home/alexd/Projekte/CAMELS/Github/camelsp/input_data/Q_and_W/BW_Baden_Wuerttemberg'

### Metadata reader

Define the function that extracts / reads and eventually merges all metadata for this federal state. You can develop the function here, without using the Bundesland context and then later use the context to pass extracted metadata. The Context has a function for saving *raw* metadata, that takes a `pandas.DataFrame` and needs you to identify the id column.
Here, *raw* refers to provider metadata, that has not yet been transformed into the CAMELS-de Metadata schema.

In [3]:
# define the function 
def read_meta(base_path) -> pd.DataFrame:
    path = os.path.join(base_path, 'BW_Meta.xlsx')
    meta = pd.read_excel(path)
    return meta

# test it here
metadata = read_meta(BASE)

metadata

,Messstellennummer,Standort,Gewässer,Ost (UTM ETRS89),Nord (UTM ETRS89),Pegelnullpunkt (PNP) in m,Pegelnullpunkt (PNP): Höhensystem,Einzugsgebiet in km²
0,105,Kirchen-Hausen,Donau,476064.18,5308070.39,"657,334",DHHN2016 (HS170),758.528
1,106,Möhringen,Donau,482087.76,5310655.37,"649,162",DHHN2016 (HS170),826.963
2,120,Hundersingen,Donau,529505.85,5324430.06,"542,53",DHHN12 (HS130),2621.324
3,125,Berg,Donau,554281.85,5346154.31,"489,903",DHHN12 (HS130),4072.790
4,129,Achstetten,Baierzer Rot,566853.80,5345973.36,"489,317",DHHN12 (HS130),264.393
...,...,...,...,...,...,...,...,...
254,76274,Erlenbach,Sulm,519268.06,5446405.15,"160,832",DHHN12 (HS130),101.510
255,76276,Bolheim,Brenz,585063.46,5386985.83,"473,0",DHHN12 (HS130),339.811
256,76290,Schweinhausen,Riß,558687.15,5320679.63,"541,098",DHHN12 (HS130),101.589
257,76361,Hölzlebruck,Josbach,439609.70,5308305.12,NaN,NaN,47.310


# Stations without data
There are some stations in the metadata for which we do not have datafiles.  
We delete them for now from the metadata!

In [4]:
ids_without_data = [1159, 3310, 3339, 32366, 60681, 76167, 76361]

# drop the ids without data from metadata
metadata = metadata[~metadata['Messstellennummer'].isin(ids_without_data)].reset_index(drop=True)
metadata

,Messstellennummer,Standort,Gewässer,Ost (UTM ETRS89),Nord (UTM ETRS89),Pegelnullpunkt (PNP) in m,Pegelnullpunkt (PNP): Höhensystem,Einzugsgebiet in km²
0,105,Kirchen-Hausen,Donau,476064.18,5308070.39,"657,334",DHHN2016 (HS170),758.528
1,106,Möhringen,Donau,482087.76,5310655.37,"649,162",DHHN2016 (HS170),826.963
2,120,Hundersingen,Donau,529505.85,5324430.06,"542,53",DHHN12 (HS130),2621.324
3,125,Berg,Donau,554281.85,5346154.31,"489,903",DHHN12 (HS130),4072.790
4,129,Achstetten,Baierzer Rot,566853.80,5345973.36,"489,317",DHHN12 (HS130),264.393
...,...,...,...,...,...,...,...,...
247,76273,Blaubeuren,Blautopf,557987.28,5362854.60,"511,871",DHHN12 (HS130),0.067
248,76274,Erlenbach,Sulm,519268.06,5446405.15,"160,832",DHHN12 (HS130),101.510
249,76276,Bolheim,Brenz,585063.46,5386985.83,"473,0",DHHN12 (HS130),339.811
250,76290,Schweinhausen,Riß,558687.15,5320679.63,"541,098",DHHN12 (HS130),101.589


In [5]:
# the id column will be Messstellennummer
id_column = 'Messstellennummer'

## file extract and parse

I'll keep the files in the zip, just because. In baWü these zips are nicely flat-packed and there is actually no need to extract the zip. Later, we might want to extract and change the code below.

In [6]:
# helper to map ids to filenames
def get_filename_mapping(zippath: str) -> Dict[str, str]:
    with zipfile.ZipFile(zippath) as z:
        return {f"{f.filename.split('-')[0]}": f.filename for f in z.filelist}

def extract_file(nr: Union[int, str], variable: str, zippath: str, not_exists = 'raise') -> pd.DataFrame:
    # get filename mapping
    fmap = get_filename_mapping(zippath)
    
    # always use string
    fname = str(nr)

    # search the file 
    if fname in fmap.values():
        fname = fname
    elif fname in fmap.keys():
        fname = fmap[fname]
    else:
        FileNotFoundError(f"nr {nr} is nothing we would expect. Use a LUBW Messstellennummer or filename in the zip")
    
    # go for the file
    with zipfile.ZipFile(zippath) as z:
        if fname not in [f.filename for f in z.filelist]:
            # TODO: here, might want to warn and return an df filled with NAN
            if not_exists == 'raise':
                raise FileNotFoundError(f"{fname} is not in {zippath}")
            else:
                return pd.DataFrame(columns=['date', variable.lower(), 'flag'])
        
        # raw content
        raw = pd.read_csv(z.open(fname), encoding='latin1', skiprows=3, sep=';', decimal=',', na_values=-999)
        
        # 'q' data
        if 'Q' in raw.columns:
            return pd.DataFrame({
                'date': [dt.strptime(_, '%d.%m.%Y') for _ in raw.Datum],
                'q': raw.Q.values,
                'flag': [_.lower().strip() == 'ja' for _ in raw['Geprüft (nein=ungeprüfte Rohdaten)']],

            })
        # w data
        else:
            return pd.DataFrame({
                'date': [dt.strptime(_, '%d.%m.%Y') for _ in raw.Datum],
                'w': raw.W.values,
                'flag': [_.lower().strip() == 'ja' for _ in raw['Geprüft (nein=ungeprüfte Rohdaten)']],

            })

# test 
df = extract_file(105, 'q', os.path.join(BASE, 'BW_Q.zip'))
df

,date,q,flag
0,1922-11-01,12.2,False
1,1922-11-02,21.9,False
2,1922-11-03,58.1,False
3,1922-11-04,57.5,False
4,1922-11-05,46.1,False
...,...,...,...
36216,2021-12-27,17.0,True
36217,2021-12-28,23.2,True
36218,2021-12-29,36.6,True
36219,2021-12-30,39.4,True


### Finally run

Now, the Q and W data can be extracted along with the metadata. The cool thing is, that all the id creation, data creation, merging and the mapping from our ids to the original ids and files is done by the context. This is helpful, as we less likely screw something up.

In [7]:
with Bundesland('Ba-Wü') as bl:
    # save the metadata
    bl.save_raw_metadata(metadata, id_column, overwrite=True)

    # for reference, call the nuts-mapping as table
    nuts_map = bl.nuts_table
    print(nuts_map.head())

    # join the path for two zips
    q_zip_path = os.path.join(bl.input_path, 'BW_Q.zip')
    w_zip_path = os.path.join(bl.input_path, 'BW_W.zip')
    
    # go for all ids
    for provider_id in tqdm(nuts_map.provider_id):
        # extract the file for this provider
        q_df = extract_file(provider_id, 'q', q_zip_path, not_exists='fill_nan')
        w_df = extract_file(provider_id, 'w', w_zip_path, not_exists='fill_nan')

        # save
        bl.save_timeseries(q_df, provider_id)
        bl.save_timeseries(w_df, provider_id)


Can't find the nuts_mapping at /home/camel/camelsp/output_data/metadata/nuts_mapping.json, returning empty mapping.
    nuts_id provider_id                              path
0  DE110000         105  ./DE1/DE110000/DE110000_data.csv
1  DE110010         106  ./DE1/DE110010/DE110010_data.csv
2  DE110020         120  ./DE1/DE110020/DE110020_data.csv
3  DE110030         125  ./DE1/DE110030/DE110030_data.csv
4  DE110040         129  ./DE1/DE110040/DE110040_data.csv


100%|██████████| 252/252 [02:03<00:00,  2.04it/s]
